### Lab -- Data Prep

Welcome to today's lab!  Today we're going to shift our attention to a more demanding dataset -- the restaurants data.  A quarter million rows, dates, and categorical data make this a more interesting, realistic use case of boosting.  

The point of today's lab will be to experiment with different encoding methods and model parameters.

**Step 1:**  Load in your dataset, and declare `X` and `y`.

**Bonus:**  If you would like, encode some of the time based data we created in previous classes.  For now, just try and extract different date parts like month, day, year, etc.  If you do not do this, you should drop the date columns before declaring `X` and `y`.

In [3]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.tree import DecisionTreeRegressor, plot_tree

df = pd.read_csv(r'/Users/lauverm/dat-11-15/ClassMaterial/Unit1/data/master.csv', parse_dates=['visit_date'])

df['month'] = df['visit_date'].dt.month
df['year']  = df['visit_date'].dt.year

df = df.fillna(0)

X = df.drop(['visit_date', 'visitors'], axis = 1)
y = df['visitors']

**Step 2:** Experiment with different encoding methods

Let's do a quick check to see how different encoding methods work out of the box on our dataset.

You're going to repeat the same process for each of `OrdinalEncoder`, `TargetEncoder`, and `OneHotEncoder` and see which one gives you the best results on our data.

**2a:** Use an `OrdinalEncoder` to transform your training set with the `fit_transform` method.

If you are confused about how the transformation is happening, try using the `mapping()` method on your category encoder to get a hang of what's going on.

In [4]:
# your code here
ore = ce.OrdinalEncoder()
X1  = ore.fit_transform(X)

**2b:** Initialize a `DecisionTreeRegressor` with a `max_depth` set to 5, check the model score to see how it performed using the `score` method.

In [5]:
# your code here
tree = DecisionTreeRegressor(max_depth = 5)
tree.fit(X1, y)
tree.score(X1, y)

0.0910418888861726

**2c:** Repeat these same steps for the `TargetEncoder` and the `OneHotEncoder`

**Important:** The `OneHotEncoder` can take awhile to fit.  If nothing happens in around 4 minutes, just cancel the process and try it again later on when you have more time.

In [8]:
# your code here
ohe = ce.OneHotEncoder()
X2  = ohe.fit_transform(X)
tree.fit(X2, y)
tree.score(X2, y)

0.09044386344858102

In [11]:
te = ce.TargetEncoder()
X3 = te.fit_transform(X, y)
tree.fit(X3, y)
tree.score(X3, y)

0.4534961233081768

**Step 3:** Look at your most important features

Similar to the previous lab, take your model's most important features and load them into a dataframe to see what's driving your results.

In [13]:
# your code here

feats = pd.DataFrame({
    'Columns': X3.columns,
    'Importance': tree.feature_importances_
}).sort_values(by = 'Importance', ascending = False)

feats

,Columns,Importance
0,id,0.901086
1,day_of_week,0.095215
2,holiday,0.002277
6,longitude,0.001422
3,genre,0.000000
4,area,0.000000
5,latitude,0.000000
7,reserve_visitors,0.000000
8,month,0.000000
9,year,0.000000


**Step 4:** Using the pipeline that was discussed in class, try and do the following:
 
 - Create a pipeline for the encoder that worked best for the previous step, and a decision tree with the same parameters that were used previously

 - Create an in-sample and out-of-sample portion for your dataset.  The in sample portion will be all rows for each restaurant for the dataset, up until the last 15.  The out-of-sample portion of your data will be the last 15 days for each restaurant.  (This same task was completed in an earlier lab, so feel free to use that as a reference if you're not sure how to do this).
 
 - Fit your model on the training set, and then score it on the test set.  Note how the two different values differ.

In [18]:
# your code here

from sklearn.pipeline import make_pipeline

df.sort_values(by = ['id', 'visit_date'], inplace = True)

train = df.groupby('id').apply(lambda x: x.iloc[:-15])
test  = df.groupby('id').apply(lambda x: x.iloc[-15:])

In [19]:
X_train = train.drop(['visit_date', 'visitors'], axis = 1)
y_train = train['visitors']

X_test = test.drop(['visit_date', 'visitors'], axis = 1)
y_test = test['visitors']

pipe = make_pipeline(te, tree)

pipe.fit(X_train, y_train)

pipe.score(X_test, y_test)

0.453061252667987